In [2]:
individualMetricsBarcelona = "../Data/Barcelona/04Stage/Metrics/Individual/IndividualnetworkMetrics.pkl"
globalMetricsBarcelona = "../Data/Barcelona/04Stage/Metrics/Global/GlobalnetworkMetrics.pkl"
scoresBarcelona = "../Data/Barcelona/03Stage/finalMetadataBarcelona.csv"

In [3]:
individualMetricsRayoVallecano = "../Data/ADRayoVallecano/04Stage/Metrics/Individual/IndividualnetworkMetrics.pkl"
globalMericsRayoVallecano = "../Data/ADRayoVallecano/04Stage/Metrics/Global/GlobalnetworkMetrics.pkl"
scoresRayoVallecano = "../Data/ADRayoVallecano/03Stage/finalMetadataRayo Vallecano.csv"

In [4]:
import pickle
import networkx as nx
import pandas as pd


In [5]:
def load_metrics(file):
    try:
        with open(file, "rb") as f:
            deserializedFile = pickle.load(f)
        return deserializedFile
    except FileNotFoundError:
        print(f"File '{individualMetricsBarcelona}' not found.")
    except nx.NetworkXError as e:
        print(f"Error reading graph from '{individualMetricsBarcelona}': {e}")

In [6]:
def load_scores(scores):
    try:
        # Open the CSV file
        # Create a CSV reader object
        dfScore = pd.read_csv(scores)
        return dfScore
    except FileNotFoundError:
        print("The file does not exist.")
    except Exception as e:
        print("An error occurred:", e)

In [7]:
def processMetrics(dfScore, deserializedFile, team):
    for score in dfScore["Score"].unique():
        metricsTable = pd.DataFrame()
        for element in deserializedFile:
            if score in deserializedFile[element]:
                df = pd.DataFrame.from_dict(deserializedFile[element][score], orient='index')        
                dfNoNan = df.fillna(" ")
                dfNoBlank = dfNoNan.replace(" ", float('nan'))  # Replace blank spaces with NaN

                # Calculate mean, standard deviation, and count
                meanValues = dfNoBlank.mean(axis=1).round(2)
                stdValues = dfNoBlank.std(axis=1).round(2)
                countValues = dfNoBlank.count(axis=1)
                metricsTable = pd.concat([metricsTable, meanValues.rename('Mean'), stdValues.rename('Std'), countValues.rename('Count')], axis=1)
        metricsTable.to_csv(f"{team}/Individual/{score}_individualMetrics.csv", index=True)


In [8]:
def filterScoresRayoVallecano(dfScore):
    # Convert 'Difference' column to numeric
    dfScore['Difference'] = pd.to_numeric(dfScore['Difference'], errors='coerce')

    # Group by the 'Score' column and calculate the size of each group
    scoreCounts = dfScore.groupby('Score').size()

    # Filter the DataFrame by the conditions: 'Difference' <= 3, size of 'Score' > 1, and 'Difference' < 0
    filteredDfScore = dfScore[( dfScore['Score'].isin(scoreCounts[scoreCounts > 1].index)) | (dfScore['Difference'] >= 0)]

    return filteredDfScore

In [9]:
def filterScoresBarcelona(dfScore):
    # Convert 'Difference' column to numeric
    dfScore['Difference'] = pd.to_numeric(dfScore['Difference'], errors='coerce')

    # Group by the 'Score' column and calculate the size of each group
    scoreCounts = dfScore.groupby('Score').size()

    # Filter the DataFrame by the conditions: 'Difference' <= 3, size of 'Score' > 1, and 'Difference' < 0
    filteredDfScore = dfScore[((dfScore['Difference'] < 3) & dfScore['Score'].isin(scoreCounts[scoreCounts > 1].index)) | (dfScore['Difference'] <= 0) | (dfScore["Score"] == "3_2") | ((dfScore["Score"] == "3_0"))]

    return filteredDfScore

In [10]:
def generateGlobalMetrics( networkMetrics, team):
    metricsTable = pd.DataFrame()
    for element in networkMetrics:
        df = pd.DataFrame.from_dict(networkMetrics[element], orient='index')
        dfNoNan = df.fillna(" ")
        dfNoBlank = dfNoNan.replace(" ", float('nan'))  # Replace blank spaces with NaN

        # Calculate mean, standard deviation, and count
        meanValues = dfNoBlank.mean(axis=1).round(2)
        stdValues = dfNoBlank.std(axis=1).round(2)
        countValues = dfNoBlank.count(axis=1)

        metricsTable = pd.concat([metricsTable, meanValues.rename('Mean'), stdValues.rename('Std'), countValues.rename('Count')], axis=1)

    metricsTable.to_csv(f"{team}/Global/globalMetrics.csv", index=True)

FCBARCELONA

In [11]:
individualMetricsBarcelonaFile = load_metrics(individualMetricsBarcelona)
globalMetricsBarcelonaFile = load_metrics(globalMetricsBarcelona)
scoresBarcelonaFile = load_scores(scoresBarcelona)

In [16]:
individualMetricsBarcelonaFile

<built-in method keys of dict object at 0x00000287EDE94200>


In [12]:
for key in individualMetricsBarcelonaFile.keys:
    print(individualMetricsBarcelonaFile[key])

TypeError: 'builtin_function_or_method' object is not iterable

In [ ]:
filteredScoresBarcelona = filterScoresBarcelona(scoresBarcelonaFile)

In [ ]:
processMetrics(filteredScoresBarcelona, individualMetricsBarcelonaFile, "Barcelona")

OSError: Cannot save file into a non-existent directory: 'Barcelona\Individual'

In [ ]:
generateGlobalMetrics(globalMetricsBarcelonaFile, "Barcelona")

RAYO VALLECANO

In [ ]:
individualMetricsRayoVallecanoFile = load_metrics(individualMetricsRayoVallecano)
globalMetricsRayoVallecanoFile = load_metrics(globalMericsRayoVallecano)
scoresFileRayoVallecano = load_scores(scoresRayoVallecano)

In [ ]:
filteredScoresRayoVallecano = filterScoresRayoVallecano(scoresFileRayoVallecano)

<bound method Series.unique of 0      0_1
1       NF
2      1_0
3      1_1
4      2_1
      ... 
159     NF
160    0_1
161    0_2
162    1_2
163     NF
Name: Score, Length: 156, dtype: object>


In [ ]:
processMetrics(filteredScoresRayoVallecano, individualMetricsRayoVallecanoFile, "RayoVallecano")

In [ ]:
generateGlobalMetrics(globalMetricsRayoVallecanoFile, "RayoVallecano")